In [1]:
import os
import json
import gc
from pathlib import Path
import numpy as np
import pandas as pd
from tqdm import tqdm
from sklearn.feature_extraction.text import TfidfVectorizer
from typing import Dict, List, Set, Tuple, NamedTuple, Callable, Any
import textstat
import scipy
import scml
from scml import pandasx as pdx
from daigt.preprocess import en as pen
from daigt.features import fit_tokenizer
from warnings import simplefilter 
simplefilter(action="ignore", category=pd.errors.PerformanceWarning)
tim = scml.Timer()
tim.start()
os.environ["TOKENIZERS_PARALLELISM"] = "false"
percentiles=[.01, .05, .1, .2, .3, .4, .5, .6, .7, .8, .9, .95, .99]
pd.set_option("use_inf_as_na", True)
pd.set_option("max_info_columns", 9999)
pd.set_option("display.max_columns", 9999)
pd.set_option("display.max_rows", 9999)
pd.set_option('max_colwidth', 9999)
tqdm.pandas()
scml.seed_everything()
info = np.finfo(np.float16)
print(f"float16, min={info.min}, max={info.max}")

float16, min=-65504.0, max=65504.0


In [2]:
df = pd.read_parquet("input/white.parquet")
df = df.drop(columns=["text", "prompt"])
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221612 entries, 0 to 221611
Data columns (total 11 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   row_id          221612 non-null  int32  
 1   essay_id        221612 non-null  object 
 2   generated       221612 non-null  int8   
 3   source          221612 non-null  object 
 4   text_bsc        221612 non-null  object 
 5   text_bow        221612 non-null  object 
 6   text_bow_len    221612 non-null  int16  
 7   prompt_bsc      221612 non-null  object 
 8   prompt_bow      221612 non-null  object 
 9   prompt_bow_len  221612 non-null  int16  
 10  white_sim       221612 non-null  float32
dtypes: float32(1), int16(2), int32(1), int8(1), object(6)
memory usage: 12.9+ MB


# Character level features

In [3]:
%%time
text_col = "text_bsc"
col = "ch_len"
df[col] = df[text_col].str.len()
df[col] = df[col].astype(np.int32)

def digit_frac(row) -> float:
    return pen.digit_frac(row[text_col])


def letter_frac(row) -> float:
    return pen.letter_frac(row[text_col])


def space_frac(row) -> float:
    return pen.space_frac(row[text_col])


def punc_frac(row) -> float:
    return pen.punc_frac(row[text_col])


def upper_frac(row) -> float:
    return pen.upper_frac(row[text_col])


def repeat_char_frac(row) -> float:
    return pen.repeat_char_frac(row[text_col])


def repeat_substring_frac(row) -> float:
    return pen.repeat_substring_frac(row[text_col])


char_fns: Dict[str, Callable] = {
    "ch_digit_frac": digit_frac,
    "ch_letter_frac": letter_frac,
    "ch_space_frac": space_frac,
    "ch_punc_frac": punc_frac,
    "ch_upper_frac": upper_frac,
    "ch_repeat_char_frac": repeat_char_frac,
    #"ch_repeat_substring_frac": repeat_substring_frac,
}

for col, fn in char_fns.items():
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(np.float32)

ch_digit_frac


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:25<00:00, 8693.03it/s]


ch_letter_frac


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:28<00:00, 7839.84it/s]


ch_space_frac


...

ch_upper_frac


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:25<00:00, 8577.45it/s]


ch_repeat_char_frac


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [01:20<00:00, 2768.32it/s]

CPU times: user 3min 31s, sys: 724 ms, total: 3min 32s
Wall time: 3min 32s


# Word and Sentence Features

In [4]:
text_col = "text_bsc"

def sent_len_mean(row) -> float:
    return pen.sentence_length_mean(row[text_col])


def sent_len_std(row) -> float:
    return pen.sentence_length_std(row[text_col])


def sent_len_delta_mean(row) -> float:
    return pen.sentence_length_delta_mean(row[text_col])


def sent_len_delta_std(row) -> float:
    return pen.sentence_length_delta_std(row[text_col])


fmap: Dict[str, Callable] = {
    "ws_sent_len_mean": sent_len_mean,
    "ws_sent_len_std": sent_len_std,
    "ws_sent_len_delta_mean": sent_len_delta_mean,
    "ws_sent_len_delta_std": sent_len_delta_std,
}
for col, fn in fmap.items():
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(np.float32)

ws_sent_len_mean


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:47<00:00, 4620.25it/s]


ws_sent_len_std


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:50<00:00, 4352.09it/s]


ws_sent_len_delta_mean


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:49<00:00, 4519.06it/s]


ws_sent_len_delta_std


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:51<00:00, 4307.25it/s]


# Textstat Features

In [5]:
text_col = "text_bsc"

def syllable_count(row) -> int:
    return textstat.syllable_count(row[text_col])


def lexicon_count(row) -> int:
    return textstat.lexicon_count(row[text_col])


def sentence_count(row) -> int:
    return textstat.sentence_count(row[text_col])


def syllables_per_word(row) -> float:
    return row["ts_syllable_count"] / (row["ts_lexicon_count"] + 1)


def syllables_per_sent(row) -> float:
    return row["ts_syllable_count"] / (row["ts_sentence_count"] + 1)


def words_per_sent(row) -> float:
    return row["ts_lexicon_count"] / (row["ts_sentence_count"] + 1)
    

def polysyllable_frac(row) -> float:
    return textstat.polysyllabcount(row[text_col]) / (row["ts_lexicon_count"] + 1)


def monosyllable_frac(row) -> float:
    return textstat.monosyllabcount(row[text_col]) / (row["ts_lexicon_count"] + 1)


def flesch_reading_ease(row) -> float:
    return textstat.flesch_reading_ease(row[text_col])


def flesch_kincaid_grade(row) -> float:
    return textstat.flesch_kincaid_grade(row[text_col])


def gunning_fog(row) -> float:
    return textstat.gunning_fog(row[text_col])


def smog_index(row) -> float:
    return textstat.smog_index(row[text_col])


def automated_readability_index(row) -> float:
    return textstat.automated_readability_index(row[text_col])


def coleman_liau_index(row) -> float:
    return textstat.coleman_liau_index(row[text_col])


def linsear_write_formula(row) -> float:
    return textstat.linsear_write_formula(row[text_col])


def dale_chall_readability_score(row) -> float:
    return textstat.dale_chall_readability_score(row[text_col])


def difficult_words(row) -> float:
    return textstat.difficult_words(row[text_col])


def spache_readability(row) -> float:
    return textstat.spache_readability(row[text_col])


def mcalpine_eflaw(row) -> float:
    return textstat.mcalpine_eflaw(row[text_col])


stage1: List[Tuple[str, Callable, Any]] = [
    ("ts_syllable_count", syllable_count, np.int32),
    ("ts_lexicon_count", lexicon_count, np.int32),
    ("ts_sentence_count", sentence_count, np.int32),
]
stage2: List[Tuple[str, Callable, Any]] = [
    ("ts_syllables_per_word", syllables_per_word, np.float32),
    ("ts_syllables_per_sent", syllables_per_sent, np.float32),
    ("ts_words_per_sent", words_per_sent, np.float32),
    ("ts_polysyllable_frac", polysyllable_frac, np.float32),
    ("ts_monosyllable_frac", monosyllable_frac, np.float32),
    ("ts_flesch_reading_ease", flesch_reading_ease, np.float32),
    ("ts_flesch_kincaid_grade", flesch_kincaid_grade, np.float32),
    ("ts_gunning_fog", gunning_fog, np.float32),
    ("ts_smog_index", smog_index, np.float32),
    ("ts_automated_readability_index", automated_readability_index, np.float32),
    ("ts_coleman_liau_index", coleman_liau_index, np.float32),
    ("ts_linsear_write_formula", linsear_write_formula, np.float32),
    ("ts_dale_chall_readability_score", dale_chall_readability_score, np.float32),
    ("ts_difficult_words", difficult_words, np.float32),
    ("ts_spache_readability", spache_readability, np.float32),
    ("ts_mcalpine_eflaw", mcalpine_eflaw, np.float32),
]
for col, fn, dtype in stage1:
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(dtype)
for col, fn, dtype in stage2:
    print(col)
    df[col] = df.progress_apply(fn, axis=1)
    df[col] = df[col].astype(dtype)

ts_syllable_count


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [01:34<00:00, 2357.18it/s]


ts_lexicon_count


100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:16<00:00, 13441.01it/s]


ts_sentence_count


...

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [01:47<00:00, 2059.54it/s]


ts_spache_readability


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [02:35<00:00, 1424.52it/s]


ts_mcalpine_eflaw


100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [01:01<00:00, 3579.50it/s]


# VAD Features

In [6]:
vdf = pd.read_csv("input/NRC-VAD-Lexicon/NRC-VAD-Lexicon.txt", header=0, names=["term", "valence", "arousal", "dominance"], 
                 sep="\t", engine="c", low_memory=False)
cols = ["valence", "arousal", "dominance"]
vdf[cols] = vdf[cols].astype(np.float32)
vdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19970 entries, 0 to 19969
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   term       19969 non-null  object 
 1   valence    19970 non-null  float32
 2   arousal    19970 non-null  float32
 3   dominance  19970 non-null  float32
dtypes: float32(3), object(1)
memory usage: 390.2+ KB


In [7]:
vad: Dict[str, Tuple] = {}
for t in tqdm(vdf.itertuples()):
    v = getattr(t, "valence")
    a = getattr(t, "arousal")
    d = getattr(t, "dominance")
    t = str(getattr(t, "term")).strip().lower()
    vad[t] = (v, a, d)

19970it [00:00, 1370333.27it/s]


In [8]:
cols = ["va_valence_mean", "va_valence_std", "va_arousal_mean", "va_arousal_std", "va_dominance_mean", "va_dominance_std"]
rows = []
for text in tqdm(df["text_bow"]):
    vas, ars, dos = [], [], []
    tokens = text.split()
    for t in tokens:
        if t in vad:
            v, a, d = vad[t]
            vas.append(v)
            ars.append(a)
            dos.append(d)
    if len(vas) == 0:
        vas = [-1]
    if len(ars) == 0:
        ars = [-1]
    if len(dos)==0:
        dos = [-1]
    rows.append([np.mean(vas), np.std(vas), np.mean(ars), np.std(ars), np.mean(dos), np.std(dos)])
df[cols] = rows
df[cols] = df[cols].astype(np.float32)
del rows, vdf, vad
gc.collect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 221612/221612 [00:45<00:00, 4878.50it/s]


32

# TF-IDF Features

In [9]:
%%time
corpus = df["text_bow"].tolist()
tokenizer = fit_tokenizer(corpus=corpus)
dp = "output/tokenizer"
Path(dp).mkdir(parents=True, exist_ok=True)
tokenizer.save_pretrained(dp)
print(f"{repr(tokenizer)}\nmodel_input_names={tokenizer.model_input_names}")




PreTrainedTokenizerFast(name_or_path='', vocab_size=30522, model_max_length=1000000000000000019884624838656, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	1: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	2: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	3: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	4: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}
model_input_names=['input_ids', 'token_type_ids', 'attention_mask']
CPU times: user 13min 22s, sys: 4.42 s, total: 

In [10]:
%%time
tokenized = [tokenizer.tokenize(x) for x in corpus]
print(tokenized[0])

['Ġcars', 'Ġcars', 'Ġhave', 'Ġbeen', 'Ġaround', 'Ġsince', 'Ġthey', 'Ġbecame', 'Ġfamous', 'Ġin', 'Ġthe', 'Ġ1900', 's', 'Ġwhen', 'Ġhenry', 'Ġford', 'Ġcreated', 'Ġand', 'Ġbuilt', 'Ġthe', 'Ġfirst', 'Ġmodel', 't', 'Ġcars', 'Ġhave', 'Ġplayed', 'Ġa', 'Ġmajor', 'Ġrole', 'Ġin', 'Ġour', 'Ġevery', 'Ġday', 'Ġlives', 'Ġsince', 'Ġthen', 'Ġbut', 'Ġnow', 'Ġpeople', 'Ġare', 'Ġstarting', 'Ġto', 'Ġquestion', 'Ġif', 'Ġlimiting', 'Ġcar', 'Ġusage', 'Ġwould', 'Ġbe', 'Ġa', 'Ġgood', 'Ġthing', 'Ġto', 'Ġme', 'Ġlimiting', 'Ġthe', 'Ġuse', 'Ġof', 'Ġcars', 'Ġmight', 'Ġbe', 'Ġa', 'Ġgood', 'Ġthing', 'Ġto', 'Ġdo', 'Ġin', 'Ġlike', 'Ġmatter', 'Ġof', 'Ġthis', 'Ġarticle', 'Ġin', 'Ġgerman', 'Ġsuburb', 'Ġlife', 'Ġgoes', 'Ġon', 'Ġwithout', 'Ġcars', 'Ġby', 'Ġelizabeth', 'Ġrosenthal', 'Ġstates', 'Ġhow', 'Ġautomobiles', 'Ġare', 'Ġthe', 'Ġlin', 'ch', 'pin', 'Ġof', 'Ġsuburbs', 'Ġwhere', 'Ġmiddle', 'Ġclass', 'Ġfamilies', 'Ġfrom', 'Ġeither', 'Ġshanghai', 'Ġor', 'Ġchicago', 'Ġtend', 'Ġto', 'Ġmake', 'Ġtheir', 'Ġhomes', 'Ġexperts', 'Ġs

In [11]:
%%time
vectorizer = TfidfVectorizer(
    ngram_range=(3, 4), 
    max_features=50_000,
    lowercase=False, 
    sublinear_tf=True, 
    analyzer='word',
    tokenizer=lambda x: x,
    preprocessor=lambda x: x,
    token_pattern=None, 
    strip_accents=None,
    dtype=np.float32,  # saves memory! scipy.sparse does not support fp16
)
x = vectorizer.fit_transform(tokenized)
print(f"x={x.shape}")

x=(221612, 50000)
CPU times: user 17min 22s, sys: 23.8 s, total: 17min 46s
Wall time: 17min 51s


In [12]:
#del vectorizer,x
#gc.collect()

In [13]:
%%time
cols = ["_".join(["tf"] + f.split()) for f in vectorizer.get_feature_names_out()]
df[cols] = scipy.sparse.csr_matrix(x).todense().astype(np.float32)
df = df.copy()  # defragment

CPU times: user 3min 35s, sys: 51.4 s, total: 4min 26s
Wall time: 4min 54s


In [14]:
vocab = {k: int(v) for k,v in vectorizer.vocabulary_.items()}
print(f"{len(vocab):,} vocab")
idf = vectorizer.idf_
print(f"idf={idf.shape}")
with open("output/tfidf.json", "w") as f:
    json.dump({"vocabulary": vocab, "idf": idf.tolist()}, f)

50,000 vocab
idf=(50000,)


In [15]:
del vectorizer, x, corpus, vocab, idf, tokenizer, tokenized
gc.collect()

32

# Review Data

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 221612 entries, 0 to 221611
Columns: 50047 entries, row_id to tf_Ġyoutube_Ġuploaded_Ġby
dtypes: float32(50033), int16(2), int32(5), int8(1), object(6)
memory usage: 41.3+ GB


In [17]:
features = []
prefixes = ["ch_", "ws_", "ts_", "va_", "tf_Ġhe"]
for col in df.columns:
    for prefix in prefixes:
        if col.startswith(prefix):
            features.append(col)
features.sort()
df[features].describe(percentiles=percentiles)

,ch_digit_frac,ch_len,ch_letter_frac,ch_punc_frac,ch_repeat_char_frac,ch_space_frac,ch_upper_frac,tf_Ġhe_Ġand_Ġhis,tf_Ġhe_Ġargued_Ġthat,tf_Ġhe_Ġargues_Ġthat,tf_Ġhe_Ġargues_Ġthat_Ġthe,tf_Ġhe_Ġasserts_Ġthat,tf_Ġhe_Ġbecame_Ġa,tf_Ġhe_Ġbelieved_Ġthat,tf_Ġhe_Ġbelieves_Ġthat,tf_Ġhe_Ġcame_Ġto,tf_Ġhe_Ġcan_Ġbe,tf_Ġhe_Ġclaimed_Ġthat,tf_Ġhe_Ġclaims_Ġthat,tf_Ġhe_Ġcould_Ġbe,tf_Ġhe_Ġcould_Ġhave,tf_Ġhe_Ġcould_Ġnot,tf_Ġhe_Ġdecided_Ġto,tf_Ġhe_Ġdecides_Ġto,tf_Ġhe_Ġdid_Ġnot,tf_Ġhe_Ġdid_Ġnot_Ġhave,tf_Ġhe_Ġdid_Ġnot_Ġwant,tf_Ġhe_Ġdoes_Ġnot,tf_Ġhe_Ġdoes_Ġnot_Ġhave,tf_Ġhe_Ġdoes_Ġnot_Ġwant,tf_Ġhe_Ġexplains_Ġthat,tf_Ġhe_Ġfailed_Ġto,tf_Ġhe_Ġfails_Ġto,tf_Ġhe_Ġhad_Ġa,tf_Ġhe_Ġhad_Ġbeen,tf_Ġhe_Ġhad_Ġno,tf_Ġhe_Ġhad_Ġnot,tf_Ġhe_Ġhad_Ġthe,tf_Ġhe_Ġhad_Ġto,tf_Ġhe_Ġhas_Ġa,tf_Ġhe_Ġhas_Ġalso,tf_Ġhe_Ġhas_Ġbeen,tf_Ġhe_Ġhas_Ġno,tf_Ġhe_Ġhas_Ġnot,tf_Ġhe_Ġhas_Ġthe,tf_Ġhe_Ġhas_Ġto,tf_Ġhe_Ġis_Ġa,tf_Ġhe_Ġis_Ġable,tf_Ġhe_Ġis_Ġable_Ġto,tf_Ġhe_Ġis_Ġalso,tf_Ġhe_Ġis_Ġan,tf_Ġhe_Ġis_Ġdoing,tf_Ġhe_Ġis_Ġgoing,tf_Ġhe_Ġis_Ġgoing_Ġto,tf_Ġhe_Ġis_Ġin,tf_Ġhe_Ġis_Ġnot,tf_Ġhe_Ġis_Ġstill,tf_Ġhe_Ġis_Ġthe,tf_Ġhe_Ġknew_Ġthat,tf_Ġhe_Ġmanaged_Ġto,tf_Ġhe_Ġneeds_Ġto,tf_Ġhe_Ġnotes_Ġthat,tf_Ġhe_Ġor_Ġshe,tf_Ġhe_Ġor_Ġshe_Ġcan,tf_Ġhe_Ġor_Ġshe_Ġdoes,tf_Ġhe_Ġor_Ġshe_Ġhas,tf_Ġhe_Ġor_Ġshe_Ġis,tf_Ġhe_Ġor_Ġshe_Ġmay,tf_Ġhe_Ġor_Ġshe_Ġmust,tf_Ġhe_Ġor_Ġshe_Ġshould,tf_Ġhe_Ġor_Ġshe_Ġwill,tf_Ġhe_Ġor_Ġshe_Ġwould,tf_Ġhe_Ġpoints_Ġout,tf_Ġhe_Ġpoints_Ġout_Ġthat,tf_Ġhe_Ġrealized_Ġthat,tf_Ġhe_Ġrealizes_Ġthat,tf_Ġhe_Ġrefers_Ġto,tf_Ġhe_Ġsaid_Ġthat,tf_Ġhe_Ġsays_Ġthat,tf_Ġhe_Ġseems_Ġto,tf_Ġhe_Ġshe_Ġcan,tf_Ġhe_Ġshe_Ġhas,tf_Ġhe_Ġshe_Ġis,tf_Ġhe_Ġshe_Ġshould,tf_Ġhe_Ġshe_Ġwill,tf_Ġhe_Ġshould_Ġbe,tf_Ġhe_Ġshould_Ġhave,tf_Ġhe_Ġstated_Ġthat,tf_Ġhe_Ġstates_Ġthat,tf_Ġhe_Ġsuggests_Ġthat,tf_Ġhe_Ġtold_Ġme,tf_Ġhe_Ġtried_Ġto,tf_Ġhe_Ġtries_Ġto,tf_Ġhe_Ġused_Ġto,tf_Ġhe_Ġuses_Ġthe,tf_Ġhe_Ġwanted_Ġto,tf_Ġhe_Ġwants_Ġto,tf_Ġhe_Ġwas_Ġa,tf_Ġhe_Ġwas_Ġable,tf_Ġhe_Ġwas_Ġable_Ġto,tf_Ġhe_Ġwas_Ġalso,tf_Ġhe_Ġwas_Ġan,tf_Ġhe_Ġwas_Ġborn,tf_Ġhe_Ġwas_Ġborn_Ġin,tf_Ġhe_Ġwas_Ġgoing,tf_Ġhe_Ġwas_Ġin,tf_Ġhe_Ġwas_Ġnot,tf_Ġhe_Ġwas_Ġthe,tf_Ġhe_Ġwas_Ġvery,tf_Ġhe_Ġwent_Ġto,tf_Ġhe_Ġwill_Ġbe,tf_Ġhe_Ġwill_Ġnot,tf_Ġhe_Ġwould_Ġbe,tf_Ġhe_Ġwould_Ġhave,tf_Ġhe_Ġwould_Ġnot,tf_Ġhead_Ġof_Ġstate,tf_Ġhead_Ġof_Ġthe,tf_Ġhealth_Ġand_Ġhuman,tf_Ġhealth_Ġand_Ġhuman_Ġservices,tf_Ġhealth_Ġand_Ġsafety,tf_Ġhealth_Ġand_Ġsocial,tf_Ġhealth_Ġand_Ġthe,tf_Ġhealth_Ġand_Ġwelfare,tf_Ġhealth_Ġand_Ġwell,tf_Ġhealth_Ġand_Ġwell_Ġbeing,tf_Ġhealth_Ġand_Ġwellbeing,tf_Ġhealth_Ġand_Ġwellness,tf_Ġhealth_Ġas_Ġwell,tf_Ġhealth_Ġcare_Ġand,tf_Ġhealth_Ġcare_Ġcosts,tf_Ġhealth_Ġcare_Ġdelivery,tf_Ġhealth_Ġcare_Ġfacilities,tf_Ġhealth_Ġcare_Ġfor,tf_Ġhealth_Ġcare_Ġin,tf_Ġhealth_Ġcare_Ġindustry,tf_Ġhealth_Ġcare_Ġis,tf_Ġhealth_Ġcare_Ġorganization,tf_Ġhealth_Ġcare_Ġorganizations,tf_Ġhealth_Ġcare_Ġprofessionals,tf_Ġhealth_Ġcare_Ġproviders,tf_Ġhealth_Ġcare_Ġsector,tf_Ġhealth_Ġcare_Ġservices,tf_Ġhealth_Ġcare_Ġsystem,tf_Ġhealth_Ġcare_Ġsystems,tf_Ġhealth_Ġcare_Ġthe,tf_Ġhealth_Ġcare_Ġto,tf_Ġhealth_Ġcare_Ġworkers,tf_Ġhealth_Ġeducation_Ġand,tf_Ġhealth_Ġeffects_Ġof,tf_Ġhealth_Ġin_Ġthe,tf_Ġhealth_Ġissues_Ġand,tf_Ġhealth_Ġit_Ġis,tf_Ġhealth_Ġneeds_Ġof,tf_Ġhealth_Ġof_Ġa,tf_Ġhealth_Ġof_Ġthe,tf_Ġhealth_Ġorganization_Ġwho,tf_Ġhealth_Ġproblems_Ġand,tf_Ġhealth_Ġservices_Ġresearch,tf_Ġhealth_Ġstatus_Ġof,tf_Ġhealthy_Ġpeople_Ġ2020,tf_Ġheart_Ġdisease_Ġand,tf_Ġheart_Ġof_Ġdarkness,tf_Ġheart_Ġof_Ġthe,tf_Ġheavily_Ġinfluenced_Ġby,tf_Ġheavily_Ġon_Ġthe,tf_Ġheight_Ġof_Ġthe,tf_Ġheld_Ġaccountable_Ġfor,tf_Ġheld_Ġby_Ġthe,tf_Ġheld_Ġin_Ġthe,tf_Ġheld_Ġresponsible_Ġfor,tf_Ġheld_Ġthat_Ġthe,tf_Ġhelp_Ġa_Ġlot,tf_Ġhelp_Ġa_Ġperson,tf_Ġhelp_Ġeach_Ġother,tf_Ġhelp_Ġfrom_Ġthe,tf_Ġhelp_Ġimprove_Ġthe,tf_Ġhelp_Ġin_Ġimproving,tf_Ġhelp_Ġin_Ġreducing,tf_Ġhelp_Ġin_Ġthe,tf_Ġhelp_Ġit_Ġto,tf_Ġhelp_Ġme_Ġin,tf_Ġhelp_Ġme_Ġto,tf_Ġhelp_Ġme_Ġwith,tf_Ġhelp_Ġof_Ġa,tf_Ġhelp_Ġof_Ġthe,tf_Ġhelp_Ġof_Ġthis,tf_Ġhelp_Ġof_Ġwhich,tf_Ġhelp_Ġother_Ġpeople,tf_Ġhelp_Ġpeople_Ġin,tf_Ġhelp_Ġpeople_Ġto,tf_Ġhelp_Ġreduce_Ġthe,tf_Ġhelp_Ġsomeone_Ġmake,tf_Ġhelp_Ġsomeone_Ġmake_Ġa,tf_Ġhelp_Ġstudents_Ġdevelop,tf_Ġhelp_Ġstu

In [18]:
%%time
df.to_parquet(f"output/features.parquet", index=False)
assert df.notna().all(axis=None)

CPU times: user 1min 35s, sys: 5.85 s, total: 1min 41s
Wall time: 1min 41s


In [19]:
tim.stop()
print(f"Total time taken {str(tim.elapsed)}")

Total time taken 1:17:40.318822
